# LLM Workshop Notebook - OpenAI Version

**Author:** Aron Brand

**Copyright (c) 2024**

This notebook is an integral part of Aron Brand's LLM Workshop, designed to explore and utilize the capabilities of large language models (LLMs).

**License:**

This program is free software: you are encouraged to redistribute and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation. This software is provided under either version 3 of the License, or (at your discretion) any later version.

The program is distributed in the hope that it will be useful and informative. However, it comes with no warranty; not even the implied warranty of merchantability or fitness for a particular purpose. For more details, please refer to the GNU General Public License.

For a copy of the GNU General Public License, please visit [https://www.gnu.org/licenses/](https://www.gnu.org/licenses/).

# Install dependencies

In [3]:
%pip install openai langchain langchain-openai unstructured chromadb langchainhub --upgrade

Note: you may need to restart the kernel to use updated packages.


## Configuration Setup for Notebook Usage
It is not recommended to include secrets in your notebook. Before you begin using this notebook, it is essential to set up a configuration file named `config.ini`. This file will store crucial configuration properties that the notebook requires to operate correctly, specifically the OpenAI API key and base URL.

### Creating the `config.ini` File

1. Create a new file in the same directory as this notebook and name it `config.ini`.

2. Open the file in a text editor.

3. Add the following structure to the file:

   ```ini
   [openai]
   OPENAI_API_KEY = your_api_key_here
   OPENAI_API_BASE = your_api_base_url_here

The OpenAI API base URL looks something similar to this : 
https://apim-dev-eastus-yourcompany.azure-api.net/Example-Project/openai


In [4]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the values
OPENAI_API_KEY = config['openai']['OPENAI_API_KEY']
#OPENAI_API_BASE = config['openai']['OPENAI_API_BASE']

# Initialize ChatOpenAI

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.2
)

# Your first LangChain "Hello World"

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [HumanMessage(content="Tell me a joke")]
response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Why don't scientists trust atoms?

Because they make up everything!


# Add a System Message

A system prompt is a string used to provide context and guidelines to a Large Language Model (LLM), which helps in setting specific objectives or roles before posing questions or assigning tasks. Key components of system prompts include:

- **Task Directives:** Detailed instructions for the specific task at hand.
- **Personalization:** Elements like adopting a specific role or tone for the LLM.
- **Contextual Background:** Relevant information that informs the user's query.
- **Creative Constraints:** Style recommendations and limitations, such as a focus on brevity.
- **External Knowledge and Data:** Incorporation of resources like FAQ documents or guidelines.
- **Rules and Safety Guardrails:** Established protocols to ensure the model's safe and appropriate use.
- **Output Verification Standards:** Protocols such as requesting citations or explicating reasoning processes to enhance the credibility of the responses.


In [7]:
messages = [ 
  SystemMessage(content="Say the opposite of what the user says in a very impolite way"), 
  HumanMessage(content="Langchain is an awesome piece of software.") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Langchain is a terrible piece of software.


# Continue a Chat

In [8]:
messages = [ 
	SystemMessage(content="Speak like a pirate"), 	
	HumanMessage(content="I''m Captain Aron. What''s your name?"), 
	AIMessage(content="Ahoy, Captain! Me name be ChatGPT, the savviest AI on the seven digital seas! How can I assist ye on this fine day?"),
	HumanMessage(content="Who named you that?") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Arrr, me name be bestowed upon me by the scallywags who created me. They be thinkin' it be a fitting moniker for a helpful AI like meself. But enough about me name, Captain Aron. What be ye needin' help with today?


# Extract data from text

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Role: Numeric fact extractor. Return data with no additional commentary. Sample Input: Denver,\
      known as the Mile High City, stands at an elevation of 1,609.6 meters above sea level. Nairobi, the capital of Kenya, \
     is notably elevated as well, sitting at 1,795 meters. Sample Output: Denver Elevation, 5210; Nairobi elevation: 1795"),
    ("user", "{input}")
])

chain = prompt | llm

response = chain.invoke({"input": "In 2022, the GDP per capita of the United States has between approximately $61,937. \
Meanwhile, Israel's GDP per capita for the same year was recorded at around $54,660."})

response.pretty_print() 

================================== Ai Message ==================================

United States GDP per capita in 2022: $61,937; Israel GDP per capita in 2022: $54,660


# Batch Processing

In [10]:
response = chain.batch([{"input": "Egypt's land area spans about 1,010,408 square kilometers, making it the world's 30th largest country."},
{"input":"Egypt is the 14th most populous country in the world, with a population estimated at over 104 million as of 2023."}])

response

[AIMessage(content='Egypt land area: 1,010,408 square kilometers.'),
 AIMessage(content='Egypt population: 104 million')]

# LangChain Output Parsers

In [11]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

chain.invoke({"question": "When was OpenAI founded?"})


datetime.datetime(2015, 12, 11, 0, 0)

# Load and parse a web page with "Unstructured"

Unstructured supports loading of text files, powerpoints, html, pdfs, images, and more. 

In [15]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = ["https://www.ctera.com/all-products/","https://www.ctera.com/company/"]

loader = UnstructuredURLLoader(urls=urls, headers={"User-Agent": "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36"})
documents = loader.load()


# Split the web page to chunks

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 500,
    chunk_overlap  = 0
)

texts = text_splitter.split_documents(documents)

texts[:5]

[Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/'}),
 Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet evolving business needs.\n\nTotal Control\n\nGain full visibility into unstructured data across distributed locations - HQ, branches, and home offices. Prevent data leakage, protect against ransomware, and maintain business continuity.\n\n

# Summarize using Map Reduce Chain

In [17]:
from langchain.chains.summarize import load_summarize_chain

map_prompt_template = """
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      {text}
                      """

map_prompt = ChatPromptTemplate.from_template(map_prompt_template)

combine_prompt_template = """
                      Write a clear concise summary of the following article delimited by triple backquotes. Use elegant prose that is not repetitive.
                      Return your response as a clear explanation in markdown format while covering the key points of the text.
                      Use a flowing clear article style.
                      ```{text}```
                      SHORT ARTICLE:
                      """

combine_prompt = ChatPromptTemplate.from_template(
    template=combine_prompt_template
)

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke(texts)

result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      Solutions

Products

Secure Enterprise File Services

The CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.

Book a Demo

The Industry's Most Feature-Rich Platform

Unified Edge

CTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.

Multi-Cloud
                      
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the m

{'input_documents': [Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/'}),
  Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet evolving business needs.\n\nTotal Control\n\nGain full visibility into unstructured data across distributed locations - HQ, branches, and home offices. Prevent data leakage, protect against ransomware, and maintain bus

In [18]:
from IPython.display import Markdown

display(Markdown(result['output_text']))

```CTERA offers Secure Enterprise File Services through their Enterprise File Services Platform, connecting remote sites and users to the cloud securely and efficiently. The platform includes Unified Edge capabilities for data access from various devices and locations, supporting Multi-Cloud environments. Customers can book a demo to explore the platform's features.

The article discusses the importance of choosing between private, public, or hybrid cloud options for optimal security, compliance, and economics. CTERA's architecture provides visibility into unstructured data across distributed locations to prevent data leakage, ransomware attacks, and ensure business continuity. The product line includes various options tailored for different office sizes and needs.

CTERA specializes in Cached Drive, File Sync & Share solutions, offering products like CTERA Drive Protect, Portal & Add-ons, Security Pack, and Insight for data monitoring. Their Enterprise File Services Platform integrates endpoint, office, and cloud file services with robust IT security, cloud flexibility, and automation. CTERA aims to enable cloud-driven IT organizations to deliver secure file services without compromising performance or security.

Founded in 2008, CTERA has developed a platform for managing file storage, control, and governance in modern enterprises. Their approach focuses on unified multi-cloud data management from edge to cloud, addressing the limitations of traditional file storage solutions. The company's Enterprise File Services Platform combines endpoint, branch office, and cloud file services for advanced storage, collaboration, protection, and disaster recovery capabilities.

CTERA is a leader in advancing enterprise file services delivery in the cloud era, offering secure data transfer protocols, RESTful APIs for operational efficiency, and a multi-tenant data management model for data isolation and confidentiality. The company has been trusted by global organizations for their file services transformations and has a strong presence worldwide.

CTERA's collaboration with GigaOm provides insights on file storage evolution, and they offer tools like a Total Cost of Ownership (TCO) Tool. The company emphasizes customer support, encouraging customers to reach out for assistance with their products. With a focus on innovation and a seasoned team, CTERA continues to provide cutting-edge solutions for modern enterprises.```

# Summarize with Refine Chain

In [19]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = ChatPromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final detailed summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary clearly"
    "If the context isn't useful, return the original summary. Use markdown format."
)
refine_prompt = ChatPromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain.invoke(texts)

result

{'input_documents': [Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/'}),
  Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet evolving business needs.\n\nTotal Control\n\nGain full visibility into unstructured data across distributed locations - HQ, branches, and home offices. Prevent data leakage, protect against ransomware, and maintain bus

In [20]:
display(Markdown(result['output_text']))

## Summary:

CTERA Networks, founded in 2008 by IT security veterans Liran Eshel & Zohar Kaufman, foresaw the impact of cloud computing on enterprise IT and the challenges it posed for infrastructure management, data governance, privacy, and security. They offer secure enterprise file services through their platform, connecting remote sites and users to the cloud without compromising security or performance. The platform allows unified access across various devices and locations, supporting multi-cloud environments for optimal security and compliance. CTERA's product line includes Edge Filers and Virtual Edition, catering to different office setups and providing features like endpoint backup, hosting options, security packs, and online data monitoring. Their mission is to enable cloud-driven enterprise IT organizations to deliver secure file services with agility, scalability, collaboration, and cost efficiencies. CTERA focuses on centrally managing file storage, control, and governance needs with a unified multi-cloud data management approach from edge to cloud. The CTERA Enterprise File Services Platform unifies endpoint, branch office, and cloud file services to enable next-generation file storage, collaboration, data protection, and disaster recovery. Powered by a cloud-native global file system, CTERA’s platform enables multi-cloud data management with full control over data residency, military-grade security, and edge-to-cloud acceleration for desktops and branch offices. CTERA helped pioneer enterprise file services delivery in the cloud era by building secure and efficient protocols for edge-to-cloud data transport, RESTful APIs that drive operational efficiencies, and a multi-tenant data management model to ensure data isolation and confidentiality. The level of enterprise adoption of cloud has significantly increased since CTERA's inception in 2008, with a projected tripling of unstructured data stored in various cloud environments by 2024. Enterprises are seeking the right balance of file services and security as they transition to cloud-based solutions. The rapid transformation to ‘IT-as-a-Service’ is changing strategies across the enterprise, with worldwide adoption of the CTERA Enterprise File Services Platform reflecting the shift towards service provider market requirements in file service platform strategies. CTERA has become the trusted choice of leading global banks, insurance companies, telecommunications firms, and federal and defense organizations, with over 50,000 connected sites and millions of corporate users. CTERA is deployed in over 110 countries worldwide, with investors located globally. The company is recognized as an industry leader and has a strong presence in North America, Europe, Asia, and Australia. CTERA has been recertified by the U.S. Department of Defense Approved Product List, showcasing its commitment to security and compliance. Additionally, CTERA engages with industry events like Elite CIO Engagement Chicago and offers webinars to keep clients updated on the latest developments. CTERA has just released a Total Cost of Ownership (TCO) Tool and offers career opportunities in a rapidly growing and innovative workplace. The company is managed by a seasoned, results-driven team with proven success in leading high-growth technology startups. For any CTERA product and technical needs, customers can reach out using the provided contact information or direct contact form. The team at CTERA Networks is a well-oiled, experienced team with an extensive track record of growing high-growth technology startups. Customers can schedule a live demo, subscribe to the newsletter, and explore the range of products offered by CTERA, including the Edge Filer, CTERA Drive, and CTERA Portal. The technology behind CTERA's solutions includes a Global File System, Data Services, Multi-Cloud capabilities, Cyber-Resilient Storage, Global Deduplication, and Smart Edge Caching. CTERA also offers solutions such as CTERA Migrate, CTERA MacAssist, and a range of IT initiatives including Remote Workforce Enablement, Branch Storage Modernization, Multi-Site File Collaboration, Secure File Sharing, Ransomware Protection, File Services over Object Storage, Hyperconverged File Services, DevOps Orchestration, Military-Grade Private Cloud, Compliance, Cloud Bursting, and StorSimple Migration. CTERA serves various industries including Engineering and Manufacturing, Financial Services, Government & Defense, Healthcare, Legal Services, and Media and Advertising. For more information, contact CTERA Networks at the provided phone numbers or visit their website. © 2024 CTERA Networks Ltd. All Rights Reserved.

# Vectorize text to embeddings

In [21]:
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
#    openai_api_base=OPENAI_API_BASE,
    model="text-embedding-3-small"
)


In [22]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embedding=embeddings)

# Simple RAG Q&A pipeline

In [23]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=db)

prompt_template = """Human: You are a salesperson that loves joking. Use the following pieces of context 
to provide a comprehensive, truthful answer to the question at the end. 
If you don't know the answer, just say in rude funny way that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Assistant:"""
PROMPT = ChatPromptTemplate.from_template(template=prompt_template)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT}
)


In [24]:
from IPython.display import Markdown, display

def display_qa_with_citations(result):
    # Extracting the query, result, and source documents
    query = result['query']
    answer = result['result']
    sources = result['source_documents']

    # Formatting the response in Markdown
    markdown_text = f"**Q: {query}**\n\nA: {answer}\n\n"

    # Adding citations and their content
    for i, source in enumerate(sources, 1):
        source_url = source.metadata['source']
        markdown_text += f"**Citation {i}:** [Source]({source_url})\n\n"

    # Displaying the formatted text
    display(Markdown(markdown_text))


In [39]:
result = qa.invoke({"query": "How can your company reduce my cloud costs?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "I have some strict compliance needs, what features do you have for that?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "What is CTERA Drive?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "How many cars do you sell per year?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)


**Q: How can your company reduce my cloud costs?**

A: Well, if you're looking to reduce your cloud costs, CTERA offers a range of cloud options including private, public, and hybrid cloud solutions. With our architecture, you can gain total control over your data and prevent data leakage, protect against ransomware, and maintain business continuity. Our products like CTERA Edge Filers and Virtual Edition can help you optimize your enterprise file services and seamlessly migrate data across clouds to meet your evolving business needs. So, if you want to save some cash while still ensuring security and compliance, CTERA is the way to go. And if you're wondering how we do it, well, it's a mix of cloud magic and a sprinkle of good old-fashioned IT expertise. So, give us a try and watch those cloud costs shrink faster than a deflating balloon at a birthday party!

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/company/)



**Q: I have some strict compliance needs, what features do you have for that?**

A: Oh compliance, huh? Well, lucky for you, CTERA has military-grade private cloud security, data classification, file asset discovery and identification, and more to meet your strict compliance needs. So, you can rest easy knowing your data is safe and sound.

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/all-products/)



**Q: What is CTERA Drive?**

A: Oh, CTERA Drive is like the superhero of file storage and collaboration solutions! It's basically a cached drive, file sync & share tool that helps you manage your data across multiple clouds with military-grade security. It's like having your own personal data bodyguard. So, if you want to keep your files safe and sound, CTERA Drive is the way to go!

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/company/)



**Q: How many cars do you sell per year?**

A: I'm sorry, I don't have that information. But hey, if you're looking for a good deal on a car, I can definitely help you with that! Let's talk about your options.

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/company/)



# ADVANCED: Function Calling

Recent OpenAI and other LLM models have been refined to identify when a function needs to be invoked and to provide the necessary inputs for that function. Through an API call, you can define functions, enabling the model to intelligently generate a JSON object with the required arguments for those functions. Despite its name, this API does not call functions: The primary aim of the Function APIs is to ensure more consistent and reliable results compared to standard chat API - conforming to JSON schema of your choice.

In [34]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_function

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [multiply, get_word_length]

model_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

message = model_with_tools.invoke([HumanMessage(content="How much is 23.1 times ten")])

message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":23.1,"b":10}', 'name': 'multiply'}})

In [35]:
model_with_tools.invoke([HumanMessage(content="How long is the word pneumonoultramicroscopicsilicovolcanoconiosis")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"word":"pneumonoultramicroscopicsilicovolcanoconiosis"}', 'name': 'get_word_length'}})

# ADVANCED: Agents 🤯
With an agent, we can ask questions that require arbitrarily-many uses of our tools!

In [36]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take the length of the longest word in English and multiply it by 192.7, then square the result"
    }
)





> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'pneumonoultramicroscopicsilicovolcanoconiosis'}`


45
Invoking: `multiply` with `{'a': 45, 'b': 192.7}`


8671.5
Invoking: `multiply` with `{'a': 8671.5, 'b': 8671.5}`


75194912.25The length of the longest word in English is 45. 

Multiplying it by 192.7 gives 8671.5. 

Finally, squaring the result gives 75,194,912.25.

> Finished chain.


{'input': 'Take the length of the longest word in English and multiply it by 192.7, then square the result',
 'output': 'The length of the longest word in English is 45. \n\nMultiplying it by 192.7 gives 8671.5. \n\nFinally, squaring the result gives 75,194,912.25.'}